<a href="https://colab.research.google.com/github/HadarMiriamIsaacson/BS-SE-24-207/blob/main/Johnson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install keyphrase-vectorizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of spacy-curated-transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 6.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached n

In [3]:
pip install transformers

In [4]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [5]:
import pandas as pd
#from keybert import KeyBERT
#from keyphrase_vectorizers import KeyphraseCountVectorizer
#from tqdm.auto import tqdm
from datasets import load_dataset
import numpy as np
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
#import matplotlib.pyplot as plt
import torch

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


import data: keybert extraction

In [6]:
data = pd.read_excel('/content/drive/MyDrive/new-20240811T082817Z-001/new/Copy of output_file(1).xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_num          100 non-null    int64 
 1   job_description  100 non-null    object
 2   keybert2         100 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


df subsets (all 13K rows)

In [7]:
#import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split

df=pd.read_excel('/content/drive/MyDrive/new-20240811T082817Z-001/new/data_holy.xlsx')
df = df.drop(['hadar', 'nisan','chatgpt','KeyBert','chatgpt','chatgpt2','judge'], axis=1)
df = df.rename(columns={'final': 'label'})
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13693 entries, 0 to 13692
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_num          13693 non-null  int64 
 1   job_description  13693 non-null  object
 2   word             13693 non-null  object
 3   label            13693 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 428.0+ KB
None


preprocessing

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

# Assuming `df` is your DataFrame loaded from the Excel file

# Group by 'job_num'
grouped_df = df.groupby('job_num')


# Use the entire dataset as your subset
total_subset_size = len(df)  # Full dataset


# Initialize an empty list to store sampled indices
sampled_indices = []

# Sample indices from each group proportionally
for name, group in grouped_df:
    # Sample all rows from each group
    group_indices = group.index.tolist()
    sampled_indices.extend(group_indices)

# Create a new DataFrame with the sampled rows
sampled_df = df.loc[sampled_indices]

In [9]:
#import pandas as pd
#import numpy as np

# Assuming train_df is your DataFrame and it has a column named 'label'

# Step 1: Extract unique labels from the 'label' column
label_list = np.unique(df['label'])

# Step 2: Create a dictionary to map labels to numerical values (if needed)
# Here, we'll map them to themselves, but you can remap as needed.
label_dict = {label: idx for idx, label in enumerate(label_list)}

# Print the mapping dictionary to verify
print("Label mapping dictionary:", label_dict)

df['label_mapped'] = df['label'].map(label_dict)

print(df.head())  # To verify the new column


Label mapping dictionary: {0: 0, 1: 1}
   job_num                                    job_description    word  label  \
0        0  The chosen Sr. Software Developer will be part...  chosen      0   
1        0  The chosen Sr. Software Developer will be part...      sr      0   
2        0  The chosen Sr. Software Developer will be part...    part      0   
3        0  The chosen Sr. Software Developer will be part...  larger      0   
4        0  The chosen Sr. Software Developer will be part...    team      0   

   label_mapped  
0             0  
1             0  
2             0  
3             0  
4             0  


In [10]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base", add_prefix_space=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
from transformers import AutoTokenizer

# Load a pre-trained tokenizer (replace 'bert-base-uncased' with the appropriate model)
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_word_in_context(row):
    # Create a concatenated input with word and context text
    word = row["word"]
    text = row["job_description"]  # Assuming job_description provides context

    # Example: "engineer [SEP] The engineer designed the system."
    input_text = word + " [SEP] " + text

    # Tokenize the combined input
    tokenized_inputs = tokenizer(input_text, truncation=True, padding='max_length', max_length=512)

    # Assign the label of the word to the tokenized sequence
    labels = [row["label_mapped"]] * len(tokenized_inputs["input_ids"])

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply the function to the entire DataFrame
tokenized_data = df.apply(tokenize_word_in_context, axis=1)

# Convert to a list of dictionaries, which can be used directly with the Hugging Face Dataset object or model training
tokenized_dataset = list(tokenized_data)

# Example output
#print(tokenized_dataset[0])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [12]:
from sklearn.model_selection import train_test_split

# Assuming your original_df has been tokenized into tokenized_dataset
train_data, temp_data = train_test_split(tokenized_dataset, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Output the sizes of each set
print(f"train_data size: {len(train_data)} rows")
print(f"temp_data set size: {len(temp_data)} rows")
print(f"val_data size: {len(val_data)} rows")
print(f"test_data set size: {len(test_data)} rows")

train_data size: 9585 rows
temp_data set size: 4108 rows
val_data size: 2054 rows
test_data set size: 2054 rows


fine tuning

In [13]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, tokenized_dataset):
        self.tokenized_dataset = tokenized_dataset

    def __len__(self):
        return len(self.tokenized_dataset)

    def __getitem__(self, idx):
        return {key: torch.tensor(val) for key, val in self.tokenized_dataset[idx].items()}

In [14]:
train_dataset = CustomDataset(train_data)
val_dataset = CustomDataset(val_data)
test_dataset = CustomDataset(test_data)


In [15]:
model_checkpoint = "distilroberta-base"
batch_size = 8

In [16]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification

# Set the random seed for reproducibility
torch.manual_seed(0)

# Load the model checkpoint and tokenizer
model_checkpoint = "distilroberta-base"  # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Load the model for token classification
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

# Move the model to the GPU
model.to('cpu')


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay

In [17]:
batch_size = 16  # Set your batch size

model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-keyword-extraction",
    evaluation_strategy="epoch",  # Corrected to `evaluation_strategy`
    save_strategy="epoch",  # Align with eval_strategy
    logging_strategy="epoch",
    learning_rate=8e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=6,
    lr_scheduler_type="linear",
    seed=0,
    load_best_model_at_end=True  # Load the best model found during training
)

In [18]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [19]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, f1_score, accuracy_score

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=-1)

    # Remove ignored index (special tokens) and flatten the output
    true_predictions = [
        p for prediction, label in zip(predictions, labels)
        for (p, l) in zip(prediction, label) if l != -100
    ]
    true_labels = [
        l for prediction, label in zip(predictions, labels)
        for (p, l) in zip(prediction, label) if l != -100
    ]

    # Calculate precision, recall, f1, and f1_macro
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true=true_labels, y_pred=true_predictions, average=None
    )
    f1_macro = f1_score(y_true=true_labels, y_pred=true_predictions, average="macro")

    # Calculate accuracy
    accuracy = accuracy_score(y_true=true_labels, y_pred=true_predictions)

    return {
        'precision': list(precision),
        'recall': list(recall),
        'f1': list(f1),
        'f1 macro': f1_macro,
        'accuracy': accuracy
    }


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, f1_score, accuracy

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=-1)

    # Remove ignored index (special tokens) and flatten the output
    true_predictions = [
        p for prediction, label in zip(predictions, labels)
        for (p, l) in zip(prediction, label) if l != -100
    ]
    true_labels = [
        l for prediction, label in zip(predictions, labels)
        for (p, l) in zip(prediction, label) if l != -100
    ]

    # Calculate precision, recall, f1, and f1_macro
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true=true_labels, y_pred=true_predictions, average=None
    )
    f1_macro = f1_score(y_true=true_labels, y_pred=true_predictions, average="macro")

    return {
        'precision': list(precision),
        'recall': list(recall),
        'f1': list(f1),
        'f1 macro': f1_macro
        'accuracy': list(acc)
    }


In [20]:
trainer = Trainer(
    model=model,  # Ensure your model is defined
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Start training
training_output = trainer.train()

# Optionally evaluate on the test dataset
test_results = trainer.evaluate(test_dataset)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,F1 macro,Accuracy
1,0.580600,0.540986,"[0.7865893336466421, 0.4399418520832332]","[0.8819081347276264, 0.27938079833984375]","[0.8315260228819834, 0.3417417921869459]",0.586634,0.731716
2,0.482000,0.373215,"[0.8853565153056506, 0.7029080952163309]","[0.9094342777237354, 0.6453361511230469]","[0.8972338906040004, 0.6728929212815]",0.785063,0.843603
3,0.344300,0.311802,"[0.9134482880911127, 0.7393903555623379]","[0.9134760558527886, 0.7393226623535156]","[0.9134621717609268, 0.7393565074084812]",0.826409,0.870065


Trainer is attempting to log a value of "[0.7865893336466421, 0.4399418520832332]" of type <class 'list'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8819081347276264, 0.27938079833984375]" of type <class 'list'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8315260228819834, 0.3417417921869459]" of type <class 'list'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8853565153056506, 0.7029080952163309]" of type <class 'list'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.90943

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,F1 macro,Accuracy
1,0.580600,0.540986,"[0.7865893336466421, 0.4399418520832332]","[0.8819081347276264, 0.27938079833984375]","[0.8315260228819834, 0.3417417921869459]",0.586634,0.731716
2,0.482000,0.373215,"[0.8853565153056506, 0.7029080952163309]","[0.9094342777237354, 0.6453361511230469]","[0.8972338906040004, 0.6728929212815]",0.785063,0.843603
3,0.344300,0.311802,"[0.9134482880911127, 0.7393903555623379]","[0.9134760558527886, 0.7393226623535156]","[0.9134621717609268, 0.7393565074084812]",0.826409,0.870065
4,0.288600,0.293903,"[0.9033329059916105, 0.8361201855956062]","[0.9549501965791181, 0.692230224609375]","[0.9284246678664952, 0.7574017843167945]",0.842913,0.889462
5,0.257400,0.272891,"[0.9308419713136926, 0.7860002876238485]","[0.9283778169584954, 0.792266845703125]","[0.9296082611778165, 0.7891211258829207]",0.859365,0.894449
6,0.244400,0.270814,"[0.9271506501251638, 0.8047488303470693]","[0.9373087406776913, 0.7781944274902344]","[0.9322020233576753, 0.7912488994216873]",0.861725,0.897646


Trainer is attempting to log a value of "[0.9033329059916105, 0.8361201855956062]" of type <class 'list'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9549501965791181, 0.692230224609375]" of type <class 'list'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9284246678664952, 0.7574017843167945]" of type <class 'list'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9308419713136926, 0.7860002876238485]" of type <class 'list'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9283778

Trainer is attempting to log a value of "[0.938613684492625, 0.8721508352959]" of type <class 'list'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9505971134196185, 0.8442566126279863]" of type <class 'list'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.9445673929884957, 0.8579770622624766]" of type <class 'list'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


Save model and tokenizer:

In [21]:
#locally:
# Save the model and tokenizer locally
save_directory = "./fine-tuned-keyword-extraction"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


('./fine-tuned-keyword-extraction/tokenizer_config.json',
 './fine-tuned-keyword-extraction/special_tokens_map.json',
 './fine-tuned-keyword-extraction/vocab.json',
 './fine-tuned-keyword-extraction/merges.txt',
 './fine-tuned-keyword-extraction/added_tokens.json',
 './fine-tuned-keyword-extraction/tokenizer.json')

In [22]:
# Load the saved model and tokenizer to verify they were saved correctly
loaded_model = AutoModelForTokenClassification.from_pretrained(save_directory)
loaded_tokenizer = AutoTokenizer.from_pretrained(save_directory)


In [23]:
# Save the state of the trainer (including optimizer, scheduler, etc.)
trainer.save_state()


In [24]:
# Save the model and tokenizer to Google Drive
drive_save_directory = '/content/drive/MyDrive/fine-tuned-keyword-extraction'
model.save_pretrained(drive_save_directory)
tokenizer.save_pretrained(drive_save_directory)


('/content/drive/MyDrive/fine-tuned-keyword-extraction/tokenizer_config.json',
 '/content/drive/MyDrive/fine-tuned-keyword-extraction/special_tokens_map.json',
 '/content/drive/MyDrive/fine-tuned-keyword-extraction/vocab.json',
 '/content/drive/MyDrive/fine-tuned-keyword-extraction/merges.txt',
 '/content/drive/MyDrive/fine-tuned-keyword-extraction/added_tokens.json',
 '/content/drive/MyDrive/fine-tuned-keyword-extraction/tokenizer.json')